# **D599 Task 1**

### 1. Environment & Import

In [20]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


# Print library versions
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("seaborn:", sns.__version__)

pandas: 2.3.1
numpy: 2.2.6
seaborn: 0.13.2


## Part 1: Variable profiling 
- list each variable
- type, subtype
- sample values

### 1. Load the dataset

In [21]:
# Loading the dataset
data_path = "C:\\Users\\user\\OneDrive\\Documents\\D599 Task\\Employee Turnover Dataset.csv"
df = pd.read_csv(data_path)

df.head()

,EmployeeNumber,Age,Tenure,Turnover,HourlyRate,HoursWeekly,CompensationType,AnnualSalary,DrivingCommuterDistance,JobRoleArea,Gender,MaritalStatus,NumCompaniesPreviouslyWorked,AnnualProfessionalDevHrs,PaycheckMethod,TextMessageOptIn
0,1,28,6,Yes,$24.37,40,Salary,50689.6,89,Research,Female,Married,3.0,7.0,Mail Check,Yes
1,2,33,2,Yes,$24.37,40,Salary,50689.6,89,Research,Female,Married,6.0,7.0,Mail Check,Yes
2,3,22,1,No,$22.52,40,Salary,46841.6,35,Information_Technology,Female,Single,1.0,8.0,Mailed Check,Yes
3,4,23,1,No,$22.52,40,Salary,46841.6,35,Information_Technology,Female,Single,3.0,NaN,Mailed Check,Yes
4,5,40,6,No,$88.77,40,Salary,284641.6,12,Sales,Prefer Not to Answer,Single,7.0,NaN,Mail Check,Yes


### 2. Data Profiling

In [ ]:
profile = []
for col in df.columns:
    s = df[col]
    # determine quantitative vs qualitative
    if pd.api.types.is_numeric_dtype(s):
        var_type = "Quantitative/Numerical"
        subtype = "Continuous" if s.nunique() > 20 else "Discrete"
    else:
        var_type = "Qualitative/Categorical"
        unique_vals = s.dropna().unique()
        # crude ordinal inference
        ordinal_keywords = {"general","medium","high","junior","senior","entry","mid","senior"}
        if len(unique_vals) <= 12 and any(isinstance(x, str) and x.strip().lower() in ordinal_keywords for x in unique_vals):
            subtype = "Ordinal (inferred)"
        else:
            subtype = "Nominal"
    sample_values = s.dropna().astype(str).unique()[:8].tolist()
    missing = int(s.isna().sum())
    profile.append({
        "variable": col,
        "data_type": var_type,
        "subtype": subtype,
        "n_unique": int(s.nunique(dropna=True)),
        "missing": missing,
        "sample_values": sample_values
    })

profile_df = pd.DataFrame(profile)
print(profile_df.to_string(index=False))


                    variable               data_type    subtype  n_unique  missing                                                                                                sample_values
              EmployeeNumber  Quantitative/Numerical Continuous     10100        0                                                                                     [1, 2, 3, 4, 5, 6, 7, 8]
                         Age  Quantitative/Numerical Continuous        41        0                                                                             [28, 33, 22, 23, 40, 45, 34, 37]
                      Tenure  Quantitative/Numerical   Discrete        20        0                                                                                    [6, 2, 1, 16, 9, 3, 8, 4]
                    Turnover Qualitative/Categorical    Nominal         2        0                                                                                                    [Yes, No]
                 HourlyRate  Qualitative

## Part 2: Data Cleaning

### 3. Inspect the data

In [23]:
# Shape of the dataset
rows, columns = df.shape
print(f"The dataset has {rows} rows and {columns} columns")

The dataset has 10199 rows and 16 columns


In [24]:
# Display column names
print("Columns:", df.columns, "\n")

Columns: Index(['EmployeeNumber', 'Age', 'Tenure', 'Turnover', 'HourlyRate ',
       'HoursWeekly', 'CompensationType', 'AnnualSalary',
       'DrivingCommuterDistance', 'JobRoleArea', 'Gender', 'MaritalStatus',
       'NumCompaniesPreviouslyWorked', 'AnnualProfessionalDevHrs',
       'PaycheckMethod', 'TextMessageOptIn'],
      dtype='object') 



In [25]:
# Descriptive statistics
print("Descriptive statistics:")

descriptive_stats = df.describe(include='all').transpose()
display(descriptive_stats)

Descriptive statistics:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
EmployeeNumber,10199.0,NaN,NaN,NaN,5001.960977,2942.709195,1.0,2451.5,5001.0,7550.5,10100.0
Age,10199.0,NaN,NaN,NaN,44.028826,10.217864,21.0,37.0,44.0,53.0,61.0
Tenure,10199.0,NaN,NaN,NaN,8.992744,5.511985,1.0,5.0,8.0,13.0,20.0
Turnover,10199,2,No,5509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HourlyRate,10199,5244,$26.28,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HoursWeekly,10199.0,NaN,NaN,NaN,40.0,0.0,40.0,40.0,40.0,40.0,40.0
CompensationType,10199,1,Salary,10199,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AnnualSalary,10199.0,NaN,NaN,NaN,120947.568526,77566.715759,-33326.4,63252.8,101566.4,153878.4,339950.4
DrivingCommuterDistance,10199.0,NaN,NaN,NaN,45.411903,54.01175,-275.0,13.0,42.0,71.0,950.0
JobRoleArea,10199,12,Research,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 4. Detect Duplicates Entries

Duplicate rows can distort results and create bias.  
- To ensure each employee record is unique, the dataset is checked for duplicates and any found are removed.

In [26]:
dup_mask = df.duplicated()
dup_count = dup_mask.sum()
print("Exact duplicate rows:", dup_count)
# See first few duplicate rows (if any)
if dup_count:
    display(df[dup_mask].head())

Exact duplicate rows: 99


,EmployeeNumber,Age,Tenure,Turnover,HourlyRate,HoursWeekly,CompensationType,AnnualSalary,DrivingCommuterDistance,JobRoleArea,Gender,MaritalStatus,NumCompaniesPreviouslyWorked,AnnualProfessionalDevHrs,PaycheckMethod,TextMessageOptIn
10100,1,28,6,Yes,$24.37,40,Salary,50689.6,89,Research,Female,Married,3.0,7.0,Mail Check,Yes
10101,2,33,2,Yes,$24.37,40,Salary,50689.6,89,Research,Female,Married,6.0,7.0,Mail Check,Yes
10102,3,22,1,No,$22.52,40,Salary,46841.6,35,Information_Technology,Female,Single,1.0,8.0,Mailed Check,Yes
10103,4,23,1,No,$22.52,40,Salary,46841.6,35,Information_Technology,Female,Single,3.0,NaN,Mailed Check,Yes
10104,5,40,6,No,$88.77,40,Salary,284641.6,12,Sales,Prefer Not to Answer,Single,7.0,NaN,Mail Check,Yes


In [27]:
# shape before dropping duplicates
print("Original shape:", df.shape)

# drop all exact duplicates
df_clean = df.drop_duplicates().reset_index(drop=True)
print("New shape after dropping duplicates:", df_clean.shape)

Original shape: (10199, 16)
New shape after dropping duplicates: (10100, 16)


### 5. Detect Missing Values

Missing values were identified using `isnull().sum()`.  
- Numeric columns are imputed with the **median**, which is robust against outliers.  
- Categorical columns are filled with the **mode** (most frequent value) or "Unknown".


In [28]:
missing_counts = df_clean.isnull().sum().sort_values(ascending=False)
display("Missing values per column: " , missing_counts)

'Missing values per column: '

TextMessageOptIn                2258
AnnualProfessionalDevHrs        1947
NumCompaniesPreviouslyWorked     663
EmployeeNumber                     0
HourlyRate                         0
Age                                0
Tenure                             0
Turnover                           0
AnnualSalary                       0
CompensationType                   0
HoursWeekly                        0
DrivingCommuterDistance            0
MaritalStatus                      0
Gender                             0
JobRoleArea                        0
PaycheckMethod                     0
dtype: int64

In [29]:
# Numeric median imputation
num_cols = df_clean.select_dtypes(include=[np.number]).columns
for col in num_cols:
    median_val = df_clean[col].median()
    df_clean[col] = df_clean[col].fillna(median_val)


# Categorical mode imputation
cat_cols = df_clean.select_dtypes(include=['object']).columns
for col in cat_cols:
    if df_clean[col].mode().size > 0:
        mode_val = df_clean[col].mode().iloc[0]
        df_clean[col] = df_clean[col].fillna(mode_val)
    else:
        df_clean[col] = df_clean[col].fillna("Unknown")

# Verify no missing values remain
final_missing_counts = df_clean.isnull().sum().sort_values(ascending=False)
display("Missing values after imputation: ", final_missing_counts)

'Missing values after imputation: '

EmployeeNumber                  0
Age                             0
Tenure                          0
Turnover                        0
HourlyRate                      0
HoursWeekly                     0
CompensationType                0
AnnualSalary                    0
DrivingCommuterDistance         0
JobRoleArea                     0
Gender                          0
MaritalStatus                   0
NumCompaniesPreviouslyWorked    0
AnnualProfessionalDevHrs        0
PaycheckMethod                  0
TextMessageOptIn                0
dtype: int64

### 6. Detect Inconsistent Entries 

Inconsistent capitalization or spacing can cause identical values to be treated differently.  

String columns were cleaned by trimming spaces and standardizing capitalization.  

For known variations, mapping was used to unify values (e.g., "mail check" → "Mailed Check").

- categorical variations
- case
- whitespace
- typos

In [30]:
# Show unique examples for each categorical column
for col in cat_cols:
    unique_vals = df_clean[col].dropna().astype(str).unique()
    print(f"\nColumn: {col} - Unique Values Count: {len(unique_vals)}")
    print(col, "->", unique_vals[:]) # show all unique values



Column: Turnover - Unique Values Count: 2
Turnover -> ['Yes' 'No']

Column: HourlyRate  - Unique Values Count: 5244
HourlyRate  -> ['$24.37 ' '$22.52 ' '$88.77 ' ... '$30.86 ' '$95.07 ' '$93.05 ']

Column: CompensationType - Unique Values Count: 1
CompensationType -> ['Salary']

Column: JobRoleArea - Unique Values Count: 12
JobRoleArea -> ['Research' 'Information_Technology' 'Sales' 'Human_Resources'
 'Laboratory' 'Manufacturing' 'Healthcare' 'Marketing'
 'InformationTechnology' 'HumanResources' 'Information Technology'
 'Human Resources']

Column: Gender - Unique Values Count: 3
Gender -> ['Female' 'Prefer Not to Answer' 'Male']

Column: MaritalStatus - Unique Values Count: 3
MaritalStatus -> ['Married' 'Single' 'Divorced']

Column: PaycheckMethod - Unique Values Count: 7
PaycheckMethod -> ['Mail Check' 'Mailed Check' 'Direct_Deposit' 'DirectDeposit'
 'Direct Deposit' 'Mail_Check' 'MailedCheck']

Column: TextMessageOptIn - Unique Values Count: 2
TextMessageOptIn -> ['Yes' 'No']


In [31]:
# Create Mapping dictionary for inconsistent entries
mapping = {
    'Mail Check': 'Mailed Check',
    'Mailed Check': 'Mailed Check',
    'Mail_Check': 'Mailed Check',
    'MailedCheck': 'Mailed Check',
    'Direct_Deposit': 'Direct Deposit',
    'DirectDeposit': 'Direct Deposit',
    'Direct Deposit': 'Direct Deposit',
}

# apply mapping to the 'PaycheckMethod' column
if 'PaycheckMethod' in df_clean.columns:
    df_clean['PaycheckMethod'] = df_clean['PaycheckMethod'].replace(mapping)

# Verify the changes
print("\nUnique values in 'PaycheckMethod' after standardization:")
df_clean['PaycheckMethod'].unique()



Unique values in 'PaycheckMethod' after standardization:


array(['Mailed Check', 'Direct Deposit'], dtype=object)

In [32]:
# For generic normalization
df_clean[cat_cols] = df_clean[cat_cols].apply(lambda col: col.str.title())

### 5. Detect Formatting Errors 

Some numeric values were stored as text containing "$" or commas (e.g., "$24.37").  
These were cleaned using a custom function that removes unwanted symbols and converts them to numeric values.

- numbers stored as strings, currency symbols

In [33]:
# Checking the data types
df_clean.dtypes

EmployeeNumber                    int64
Age                               int64
Tenure                            int64
Turnover                         object
HourlyRate                       object
HoursWeekly                       int64
CompensationType                 object
AnnualSalary                    float64
DrivingCommuterDistance           int64
JobRoleArea                      object
Gender                           object
MaritalStatus                    object
NumCompaniesPreviouslyWorked    float64
AnnualProfessionalDevHrs        float64
PaycheckMethod                   object
TextMessageOptIn                 object
dtype: object

In [34]:
# Clean currency symbols in HourlyRate (handles variants with/without trailing space)
hourly_cols = [c for c in df_clean.columns if c.strip() == 'HourlyRate']

if not hourly_cols:
    print("No HourlyRate column found.")
else:
    for col in hourly_cols:
        # keep a backup of original strings
        backup_col = f"{col}_orig"
        if backup_col not in df_clean.columns:
            df_clean[backup_col] = df_clean[col].astype(str)
        
        # remove $ and commas, trim whitespace, convert to numeric
        cleaned = df_clean[col].astype(str).str.replace(r'[\$,]', '', regex=True).str.strip()
        df_clean[col] = pd.to_numeric(cleaned.replace('', np.nan), errors='coerce')
        
        # report how many became NaN during conversion
        n_coerced = df_clean[col].isna().sum()
        print(f"Column '{col}': {n_coerced} non-numeric/missing after stripping currency (of {len(df_clean)})")
        
        # if many missing, impute with median (consistent with previous numeric imputation)
        if n_coerced:
            med = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(med)
            print(f"Imputed {n_coerced} values in '{col}' with median = {med:.2f}")
        
        # final dtype and a quick peek
        print(f"'{col}' dtype after cleaning:", df_clean[col].dtype)
        display(df_clean[[backup_col, col]].head(6))

Column 'HourlyRate ': 0 non-numeric/missing after stripping currency (of 10100)
'HourlyRate ' dtype after cleaning: float64


,HourlyRate _orig,HourlyRate
0,$24.37,24.37
1,$24.37,24.37
2,$22.52,22.52
3,$22.52,22.52
4,$88.77,88.77
5,$88.77,88.77


### 6. Detect outliers (IQR method) and handling

Outliers were detected using the **Interquartile Range (IQR)** method.  
- Values outside 1.5 × IQR were capped at the lower or upper bound (winsorized) to reduce their effect while preserving data points.


In [35]:
# Detect outliers in df_clean using the IQR method

# identify numeric columns
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()

summary = []
for col in numeric_cols:
    s = df_clean[col].dropna()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    out_mask = (df_clean[col] < lower) | (df_clean[col] > upper)
    n_out = int(out_mask.sum())
    pct_out = n_out / len(df_clean) * 100

    summary.append({
        "variable": col,
        "n_outliers": n_out,
        "pct_outliers": pct_out,
        "lower_bound": lower,
        "upper_bound": upper,
        "iqr": iqr
    })

    # print brief info and show a sample of outliers if any
    if n_out:
        print(f"{col}: {n_out} outliers ({pct_out:.2f}%) — LB={lower:.2f}, UB={upper:.2f}")
    else:
        print(f"{col}: 0 outliers")

# Summary dataframe
outlier_summary = pd.DataFrame(summary).set_index("variable")
display(outlier_summary)

EmployeeNumber: 0 outliers
Age: 0 outliers
Tenure: 0 outliers
HourlyRate : 0 outliers
HoursWeekly: 0 outliers
AnnualSalary: 571 outliers (5.65%) — LB=-71975.80, UB=289133.00
DrivingCommuterDistance: 241 outliers (2.39%) — LB=-74.00, UB=158.00
NumCompaniesPreviouslyWorked: 0 outliers
AnnualProfessionalDevHrs: 0 outliers


,n_outliers,pct_outliers,lower_bound,upper_bound,iqr
variable,,,,,
EmployeeNumber,0,0.000000,-5048.50000,15149.50000,5049.5000
Age,0,0.000000,13.00000,77.00000,16.0000
Tenure,0,0.000000,-7.00000,25.00000,8.0000
HourlyRate,0,0.000000,-33.15375,138.13625,42.8225
HoursWeekly,0,0.000000,40.00000,40.00000,0.0000
AnnualSalary,571,5.653465,-71975.80000,289133.00000,90277.2000
DrivingCommuterDistance,241,2.386139,-74.00000,158.00000,58.0000
NumCompaniesPreviouslyWorked,0,0.000000,-4.00000,12.00000,4.0000
AnnualProfessionalDevHrs,0,0.000000,-1.00000,31.00000,8.0000


In [36]:
# Handle outliers by capping them at the IQR bounds
for col in numeric_cols:
    lower = outlier_summary.loc[col, "lower_bound"]
    upper = outlier_summary.loc[col, "upper_bound"]
    df_clean[col] = np.where(df_clean[col] < lower, lower,
                             np.where(df_clean[col] > upper, upper, df_clean[col]))
    
df_clean.head()

,EmployeeNumber,Age,Tenure,Turnover,HourlyRate,HoursWeekly,CompensationType,AnnualSalary,DrivingCommuterDistance,JobRoleArea,Gender,MaritalStatus,NumCompaniesPreviouslyWorked,AnnualProfessionalDevHrs,PaycheckMethod,TextMessageOptIn,HourlyRate _orig
0,1.0,28.0,6.0,Yes,24.37,40.0,Salary,50689.6,89.0,Research,Female,Married,3.0,7.0,Mailed Check,Yes,$24.37
1,2.0,33.0,2.0,Yes,24.37,40.0,Salary,50689.6,89.0,Research,Female,Married,6.0,7.0,Mailed Check,Yes,$24.37
2,3.0,22.0,1.0,No,22.52,40.0,Salary,46841.6,35.0,Information_Technology,Female,Single,1.0,8.0,Mailed Check,Yes,$22.52
3,4.0,23.0,1.0,No,22.52,40.0,Salary,46841.6,35.0,Information_Technology,Female,Single,3.0,15.0,Mailed Check,Yes,$22.52
4,5.0,40.0,6.0,No,88.77,40.0,Salary,284641.6,12.0,Sales,Prefer Not To Answer,Single,7.0,15.0,Mailed Check,Yes,$88.77


### 7. Recalculate Annual Salary (Data Integrity Check)

Derived Field Validation
- The Annual Salary column was recalculated as Hourly Rate × Hours Weekly × 52.  
- If the computed value differed by more than 1%, the Annual Salary was replaced.


In [37]:
if all(col in df.columns for col in ['Hourly Rate', 'Hours Weekly', 'Annual Salary']):
    computed = df['Hourly Rate'] * df['Hours Weekly'] * 52
    diff = (df['Annual Salary'] - computed).abs()
    mask = diff > (0.01 * computed)
    df.loc[mask, 'Annual Salary'] = computed[mask]
    print("Recomputed Annual Salary for", mask.sum(), "records.")


### 8. Final checks: data types, missing, sample, save cleaned CSV

- After all cleaning steps, the dataset was checked for remaining missing values, verified column types, and saved as the cleaned dataset ready for analysis.


In [38]:
# Final checks
print("Final shape:", df_clean.shape, "\n")
display("Missing counts (final): ", df_clean.isnull().sum())
display("Dtypes: ", df_clean.dtypes)
display("Sample cleaned rows: ", df_clean.head(5))



# Save cleaned file
df_clean.to_csv("Employee_Turnover_Cleaned.csv", index=False)
print("Saved cleaned dataset to Employee_Turnover_Cleaned.csv")

Final shape: (10100, 17) 



'Missing counts (final): '

EmployeeNumber                  0
Age                             0
Tenure                          0
Turnover                        0
HourlyRate                      0
HoursWeekly                     0
CompensationType                0
AnnualSalary                    0
DrivingCommuterDistance         0
JobRoleArea                     0
Gender                          0
MaritalStatus                   0
NumCompaniesPreviouslyWorked    0
AnnualProfessionalDevHrs        0
PaycheckMethod                  0
TextMessageOptIn                0
HourlyRate _orig                0
dtype: int64

'Dtypes: '

EmployeeNumber                  float64
Age                             float64
Tenure                          float64
Turnover                         object
HourlyRate                      float64
HoursWeekly                     float64
CompensationType                 object
AnnualSalary                    float64
DrivingCommuterDistance         float64
JobRoleArea                      object
Gender                           object
MaritalStatus                    object
NumCompaniesPreviouslyWorked    float64
AnnualProfessionalDevHrs        float64
PaycheckMethod                   object
TextMessageOptIn                 object
HourlyRate _orig                 object
dtype: object

'Sample cleaned rows: '

,EmployeeNumber,Age,Tenure,Turnover,HourlyRate,HoursWeekly,CompensationType,AnnualSalary,DrivingCommuterDistance,JobRoleArea,Gender,MaritalStatus,NumCompaniesPreviouslyWorked,AnnualProfessionalDevHrs,PaycheckMethod,TextMessageOptIn,HourlyRate _orig
0,1.0,28.0,6.0,Yes,24.37,40.0,Salary,50689.6,89.0,Research,Female,Married,3.0,7.0,Mailed Check,Yes,$24.37
1,2.0,33.0,2.0,Yes,24.37,40.0,Salary,50689.6,89.0,Research,Female,Married,6.0,7.0,Mailed Check,Yes,$24.37
2,3.0,22.0,1.0,No,22.52,40.0,Salary,46841.6,35.0,Information_Technology,Female,Single,1.0,8.0,Mailed Check,Yes,$22.52
3,4.0,23.0,1.0,No,22.52,40.0,Salary,46841.6,35.0,Information_Technology,Female,Single,3.0,15.0,Mailed Check,Yes,$22.52
4,5.0,40.0,6.0,No,88.77,40.0,Salary,284641.6,12.0,Sales,Prefer Not To Answer,Single,7.0,15.0,Mailed Check,Yes,$88.77


Saved cleaned dataset to Employee_Turnover_Cleaned.csv


## Summary of Part II – Data Cleaning and Plan

**Key Improvements**
- Removed duplicate rows.
- Imputed missing values using median (numeric) and mode (categorical).
- Normalized categorical variables (trimmed spaces, fixed capitalization).
- Cleaned formatting errors in salary and rate columns.
- Winsorized numeric columns to control extreme values.
- Verified calculated fields for internal consistency.

**Advantages**
1. Cleaning is fully reproducible in Python.  
2. Median and IQR handling makes the data more robust to outliers.

**Limitations**
1. Median/mode imputation can introduce bias.  
2. Winsorization keeps all data points but may mask extreme cases.

**Next Steps**
Proceed to Part III (Panopto demonstration and submission).

## References

- McKinney, W. (2022). Python for Data Analysis: Data Wrangling with pandas, NumPy, and Jupyter (3rd ed.). O’Reilly Media.
    
    Used for pandas functions like drop_duplicates(), fillna(), and to_numeric().

- Pandas Development Team. (2024). pandas Documentation. Retrieved from https://pandas.pydata.org/docs/
    
    Official documentation for data manipulation and cleaning in Python.

- Waskom, M. (2021). Seaborn: Statistical Data Visualization. Journal of Open Source Software, 6(60), 3021.
    
    Referenced for understanding statistical visualization and outlier detection.

- Statology. (2023). How to Winsorize Data in Python (With Examples). Retrieved from https://www.statology.org/winsorize-data-python/
    
    Used to explain and apply Winsorization for outlier handling.

- WGU Data Management Course Resources (2024). D599 – Data Cleaning and Preparation Guidelines. Western Governors University.
    
    Used to align the cleaning steps and documentation format with course expectations.